### 동적 스크레이핑으로 모든 도서의 링크만 가져오기
- 유효하지 않은 3개의 링크는 추후 json 파일 편집으로 제거함 ( 링크 대신 javascript:void(0); 이 기록된 경우)

In [1]:
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium import webdriver

from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC

import time
import json

# WebDriver 설정
driver = webdriver.Chrome()
driver.get('https://www.yes24.com/Product/Category/Display/001001047001')
time.sleep(2)  # 페이지 로드 대기

# 판매량 순 버튼 클릭
sales_rank_button = driver.find_element(By.CSS_SELECTOR, "#categoryProductContentsWrap > div.cateGoodsSecArea > div > span.baseFilter > a:nth-child(2)")
sales_rank_button.click()
time.sleep(2)  # 변경 적용 대기

# 책 보기 개수 설정 (120개)
page_size_dropdown = Select(driver.find_element(By.CSS_SELECTOR, "#pg_size"))
page_size_dropdown.select_by_value("120")  # 120개 선택
time.sleep(2)  # 변경 적용 대기



# 스크롤 다운 함수
def scroll_down(driver, scroll_count):
    for _ in range(scroll_count):
        driver.execute_script("window.scrollBy(0, 1000);")  # 스크롤 내리기
        # time.sleep(0.5)  # 대기

# 메인 페이지 스크롤
main_scroll_cnts = 9

# 페이지 번호 클릭 -> 이동
Total_pages_num = 85
book_links = []

#categoryProductContentsWrap > div.newGoodsPagen > div > div > div > strong
#categoryProductContentsWrap > div.newGoodsPagen > div > div > div > a:nth-child(2)
#categoryProductContentsWrap > div.newGoodsPagen > div > div > div > strong  #10번째
#categoryProductContentsWrap > div.newGoodsPagen > div > div > div > a.bgYUI.next  #다음 버튼


#categoryProductContentsWrap > div.newGoodsPagen > div > div > div > strong
#categoryProductContentsWrap > div.newGoodsPagen > div > div > div > a:nth-child(4)



for page_num in range(1, Total_pages_num+1):
    try:
        order_by_quantity = driver.find_element(By.CSS_SELECTOR, '#categoryProductContentsWrap > div.cateGoodsSecArea > div > span.baseFilter > a:nth-child(2)')
        order_by_quantity.click()

        scroll_down(driver, main_scroll_cnts)
        if page_num % 10 == 1:
            page_link = WebDriverWait(driver, 3).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, '#categoryProductContentsWrap > div.newGoodsPagen > div > div > div > strong')))
            page_link.click()
            print(page_num)
            time.sleep(1)
    
        elif 2 <= page_num < 10:
            page_link = WebDriverWait(driver, 3).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, f'#categoryProductContentsWrap .newGoodsPagen div div a:nth-child({page_num})')))
            page_link.click()
            print(page_num)
            time.sleep(1)

        elif page_num % 10 == 0:

            page_link = WebDriverWait(driver, 3).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, '#categoryProductContentsWrap > div.newGoodsPagen > div > div > div > strong')))
            page_link.click()
            print(page_num)
            time.sleep(1)

            scroll_down(driver, main_scroll_cnts)

            next_button = WebDriverWait(driver, 3).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR,'#categoryProductContentsWrap > div.newGoodsPagen > div > div > div > a.bgYUI.next')))
            next_button.click()
            print(f"{page_num} 'next button'")
            time.sleep(1)

        else:
            page_link = WebDriverWait(driver, 3).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, f'#categoryProductContentsWrap .newGoodsPagen div div a:nth-child({ (page_num % 10) + 2 })')))
            page_link.click()
            print(page_num)
            time.sleep(1)

        # 책 링크 수집
        books = driver.find_elements(By.CSS_SELECTOR, 'a.gd_name')  # 책 제목 링크
        for book in books:
            book_links.append(book.get_attribute('href'))
        print(f"수집한 링크 개수: {len(book_links)}")

    except Exception as e:
        print(f"오류 발생 (페이지 번호 {page_num}): {e}")
        continue

# ## 페이지 html 리스트#1_10
# categoryProductContentsWrap > div.newGoodsPagen > div > div > div > strong
# categoryProductContentsWrap > div.newGoodsPagen > div > div > div > a:nth-child(2)
# categoryProductContentsWrap > div.newGoodsPagen > div > div > div > a:nth-child(10)
# categoryProductContentsWrap > div.newGoodsPagen > div > div > div > a.bgYUI.next
# categoryProductContentsWrap > div.newGoodsPagen > div > div > div > a.bgYUI.end

# 11~20
# categoryProductContentsWrap > div.newGoodsPagen > div > div > div > a.bgYUI.first
# categoryProductContentsWrap > div.newGoodsPagen > div > div > div > a.bgYUI.prev
# categoryProductContentsWrap > div.newGoodsPagen > div > div > div > strong
# categoryProductContentsWrap > div.newGoodsPagen > div > div > div > a:nth-child(4)
# categoryProductContentsWrap > div.newGoodsPagen > div > div > div > a:nth-child(5) .....
# categoryProductContentsWrap > div.newGoodsPagen > div > div > div > a:nth-child(11)
# categoryProductContentsWrap > div.newGoodsPagen > div > div > div > strong         ## 20번 페이지


# 21~30
# categoryProductContentsWrap > div.newGoodsPagen > div > div > div > a.bgYUI.first
# categoryProductContentsWrap > div.newGoodsPagen > div > div > div > a.bgYUI.prev
# categoryProductContentsWrap > div.newGoodsPagen > div > div > div > strong
# categoryProductContentsWrap > div.newGoodsPagen > div > div > div > a:nth-child(4)
# categoryProductContentsWrap > div.newGoodsPagen > div > div > div > a:nth-child(5)

# next button
# categoryProductContentsWrap > div.newGoodsPagen > div > div > div > a.bgYUI.next
# categoryProductContentsWrap > div.newGoodsPagen > div > div > div > a.bgYUI.next

    



1
수집한 링크 개수: 24
2
수집한 링크 개수: 48
3
수집한 링크 개수: 72
오류 발생 (페이지 번호 4): Message: stale element reference: stale element not found in the current frame
  (Session info: chrome=133.0.6943.53); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
0   chromedriver                        0x00000001053bb94c cxxbridge1$str$ptr + 2728228
1   chromedriver                        0x00000001053b3fa8 cxxbridge1$str$ptr + 2697088
2   chromedriver                        0x0000000104f19ff0 cxxbridge1$string$len + 93348
3   chromedriver                        0x0000000104f1fcfc cxxbridge1$string$len + 117168
4   chromedriver                        0x0000000104f21dd4 cxxbridge1$string$len + 125576
5   chromedriver                        0x0000000104f21e7c cxxbridge1$string$len + 125744
6   chromedriver                        0x0000000104f6673c cxxbridge1$string$len + 406512
7   chromedriver         

In [1]:
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium import webdriver

from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC

import time
import json

# WebDriver 설정
driver = webdriver.Chrome()
driver.get('https://www.yes24.com/Product/Category/Display/001001047001')
time.sleep(2)  # 페이지 로드 대기

# 판매량 순 버튼 클릭
sales_rank_button = driver.find_element(By.CSS_SELECTOR, "#categoryProductContentsWrap > div.cateGoodsSecArea > div > span.baseFilter > a:nth-child(2)")
sales_rank_button.click()
time.sleep(2)  # 변경 적용 대기

# 책 보기 개수 설정 (120개)
page_size_dropdown = Select(driver.find_element(By.CSS_SELECTOR, "#pg_size"))
page_size_dropdown.select_by_value("120")  # 120개 선택
time.sleep(2)  # 변경 적용 대기

In [90]:
# 페이지 1745 / 수집한 링크 개수: 40872
len(book_links)

40872

In [92]:
with open('all_book_links.json', 'w', encoding='utf-8') as f:
    json.dump(book_links, f, ensure_ascii=False, indent=4)
print("모든 에세이 도서의 링크 수집 완료!")

모든 에세이 도서의 링크 수집 완료!


# 정적 웹스크레이핑

In [3]:
with open('author_loss_idx.json', 'r', encoding='utf-8') as file:
    author_loss_idx = json.load(file)
print(author_loss_idx[-10:])


[14180, 14182, 14184, 14641, 14752, 14753, 15277, 15278, 15283, 16106]


In [ ]:
import json
import re
from datetime import datetime
import requests
from bs4 import BeautifulSoup


with open('all_book_links.json', 'r', encoding='utf-8') as file:
    all_book_links = json.load(file)

all_book_data = []
invalid_url_cnt = 0
invalid_url_idx = []
img_url_error_count = 0
img_url_error_idx = []
isbn13_error_count = 0
isbn13_error_idx = []
date_error_count = 0
date_error_idx = []

# test_link = ['https://www.yes24.com/Product/Goods/1392637']
# for idx, link in enumerate(test_link):



for idx, link in enumerate(all_book_links):

    if not link.startswith('http'):
        print(f"Invalid URL skipped : {link}")
        invalid_url_cnt += 1
        invalid_url_idx.append(idx)
        continue

    response = requests.get(link)
    
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')

    ## 줄바꿈으로 부제도 표현됨.
    titles = soup.select('h2.gd_name')
    title_text = '\n'.join(set([title.get_text(strip=True) for title in titles]))

    # 도서별 책소개의 css는 총 2종류이다
    descriptions = soup.select('#infoset_introduce > div.infoSetCont_wrap > div.infoWrap_txt > div')
    if len(descriptions) == 0:
        descriptions = soup.select('#infoset_introduce > div.infoSetCont_wrap > table > tbody > tr > td > div > div')
    descriptions_text = '\n'.join([desc.get_text(strip=True) for desc in descriptions])

    key_sentences = soup.select('#infoset_inBook > div.infoSetCont_wrap > div.infoWrap_txt > div')
    key_sentences_text = '\n'.join([key_sentence.get_text(strip=True) for key_sentence in key_sentences])

    if descriptions_text and key_sentences_text:
        authors = soup.select('#yDetailTopWrap > div.topColRgt > div.gd_infoTop > span.gd_pubArea > span.gd_auth')
        authors_text = ', '.join([author.get_text(strip=True, separator=' ') for author in authors])        

        publishers = soup.select('#yDetailTopWrap > div.topColRgt > div.gd_infoTop > span.gd_pubArea > span.gd_pub > a')
        publishers_text = ', '.join([publisher.get_text(strip=True) for publisher in publishers])
        
        isbn13_element = soup.select('#infoset_specific > div.infoSetCont_wrap > div > table > tbody > tr')
        if len(isbn13_element) > 1:
            isbn13 = isbn13_element[-2]
            if isbn13:
                isbn13_text = isbn13.get_text(strip=True)
                isbn13_num_only = re.sub(r'[^0-9]', '', isbn13_text)
                try:
                    isbn13_int = int(isbn13_num_only)
                except ValueError:
                    print(f"Invalid ISBN data: {isbn13_num_only}")
                    isbn13_int = isbn13_text
                    isbn13_error_count += 1
                    isbn13_error_idx.append(idx)
            else:
                isbn13 = None
        else:
            isbn13_int = None
            
        img_url = soup.select_one('img.gImg')['src']
        if img_url is None:
            img_url_error_count += 1
            img_url_error_idx.append(idx)

        publication_date = soup.select_one('span.gd_date')
        if publication_date is not None:
            publication_date_text = publication_date.text
            parsed_date = datetime.strptime(publication_date_text, "%Y년 %m월 %d일")
            formatted_date = parsed_date.strftime("%Y-%m-%d")
        else:
            print(f"Invalid Publication_date: {idx}th book")
            formatted_date = None
            date_error_count += 1
            date_error_idx.append(idx)


        all_book_data.append({
                'title': title_text,
                'description': descriptions_text,
                'key_sentences': key_sentences_text,
                'author': authors_text,
                'publisher': publishers_text,
                'isbn13': isbn13_int,
                'img_url': img_url,
                'publication_date': formatted_date
            })
        
    print(f'{idx}-th completed')
    

with open('all_book_data_ver4.json', 'w', encoding='utf-8') as f:
    json.dump(all_book_data, f, ensure_ascii=False, indent=4)


In [ ]:
print(len(all_book_data))

print(f"{invalid_url_cnt = }")
print(f'{invalid_url_idx = }')
print(f'{img_url_error_count = }')
print(f"{img_url_error_idx = }")
print(f"{isbn13_error_count = }")
print(f"{isbn13_error_idx = }")
print(f"{date_error_count = }")
print(f"{date_error_idx = }")


In [8]:
import json

# 분할된 데이터들 병합
with open('all_book_data_ver3[0:7601].json') as file:
    book_data_pt1 = json.load(file)
print(len(book_data_pt1))

with open('all_book_data_ver3[7601:11777].json') as file:
    book_data_pt2 = json.load(file)
print(len(book_data_pt2))

with open('all_book_data_ver3[11777:14921].json') as file:
    book_data_pt3 = json.load(file)
print(len(book_data_pt3))

with open('all_book_data_ver3[14922:20000].json') as file:
    book_data_pt4 = json.load(file)
print(len(book_data_pt4))

book_data_merged = book_data_pt1 + book_data_pt2 + book_data_pt3 + book_data_pt4
print(len(book_data_merged))
    

4885
1431
1025
1566
8907


In [10]:
# 병합된 데이터들 파일로 저장
with open('all_book_data_ver3[0:20000].json', 'w', encoding='utf-8') as f:
    json.dump(book_data_merged, f, ensure_ascii=False, indent=4)


In [3]:
import json
with open('all_book_data_ver3[0:20000].json', 'r', encoding='utf-8') as file:
    book_data_pt5 = json.load(file)
print(len(book_data_pt5))

8907


In [4]:

with open('all_book_data_ver3[20000:].json', 'r', encoding='utf-8') as file:
    book_data_pt6 = json.load(file)
print(len(book_data_pt6))

7490


In [5]:
all_book_merged_data = book_data_pt5 + book_data_pt6

print(len(all_book_merged_data))

with open('all_book_data_ver3.json', 'w', encoding='utf-8') as f:
    json.dump(all_book_merged_data, f, ensure_ascii=False, indent=4)

with open('all_book_data_ver3.json', 'r', encoding='utf-8') as file2:
    test_len = json.load(file2)
    
print(len(test_len))

16397
16397


In [1]:
import json
with open('all_book_data_ver3 copy.json', 'r', encoding='utf-8') as file:
    set_book_data = json.load(file)

print(len(set_book_data))

16397


In [2]:

# 딕셔너리를 문자열로 변환하여 중복 제거
unique_dicts = list({json.dumps(d, sort_keys=True) for d in set_book_data})
unique_dicts = [json.loads(d) for d in unique_dicts]

print(len(unique_dicts))

14390
